# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 22 2023 8:30AM,257885,10,0086417262,ISDIN Corporation,Released
1,Feb 22 2023 8:30AM,257885,10,0086417271,ISDIN Corporation,Released
2,Feb 22 2023 8:30AM,257885,10,0086417273,ISDIN Corporation,Released
3,Feb 22 2023 8:30AM,257885,10,0086417290,ISDIN Corporation,Released
4,Feb 22 2023 8:30AM,257885,10,0086417295,ISDIN Corporation,Released
5,Feb 22 2023 8:30AM,257885,10,0086417301,ISDIN Corporation,Released
6,Feb 22 2023 8:30AM,257885,10,0086417211,ISDIN Corporation,Released
7,Feb 22 2023 8:30AM,257885,10,0086417223,ISDIN Corporation,Released
8,Feb 22 2023 8:30AM,257885,10,0086417236,ISDIN Corporation,Released
9,Feb 22 2023 8:30AM,257885,10,0086417242,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,257879,Released,38
49,257880,Released,6
50,257881,Released,17
51,257884,Released,2
52,257885,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257879,NaN,NaN,38.0
257880,NaN,NaN,6.0
257881,NaN,NaN,17.0
257884,NaN,NaN,2.0
257885,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257739,13.0,5.0,1.0
257797,0.0,0.0,33.0
257798,0.0,0.0,26.0
257799,0.0,0.0,32.0
257800,0.0,0.0,45.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257739,13,5,1
257797,0,0,33
257798,0,0,26
257799,0,0,32
257800,0,0,45


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257739,13,5,1
1,257797,0,0,33
2,257798,0,0,26
3,257799,0,0,32
4,257800,0,0,45


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257739,13,5,1
1,257797,,,33
2,257798,,,26
3,257799,,,32
4,257800,,,45


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation
12,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation
48,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation
84,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc."
98,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc."
108,Feb 22 2023 8:30AM,257884,10,"Snap Medical Industries, LLC"
110,Feb 22 2023 8:30AM,257880,10,Methapharm-G
111,Feb 21 2023 2:57PM,257870,10,ISDIN Corporation
164,Feb 21 2023 2:53PM,257877,10,ISDIN Corporation
170,Feb 21 2023 2:51PM,257874,20,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation,,,14
1,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation,,,38
2,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation,,,34
3,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc.",,,17
4,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc.",,,6
5,Feb 22 2023 8:30AM,257880,10,Methapharm-G,,,6
6,Feb 22 2023 8:30AM,257884,10,"Snap Medical Industries, LLC",,,2
7,Feb 21 2023 2:57PM,257870,10,ISDIN Corporation,,,53
8,Feb 21 2023 2:53PM,257877,10,ISDIN Corporation,,,6
9,Feb 21 2023 2:51PM,257874,20,"Emersa Waterbox, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation,14,,
1,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation,38,,
2,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation,34,,
3,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc.",17,,
4,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc.",6,,
5,Feb 22 2023 8:30AM,257880,10,Methapharm-G,6,,
6,Feb 22 2023 8:30AM,257884,10,"Snap Medical Industries, LLC",2,,
7,Feb 21 2023 2:57PM,257870,10,ISDIN Corporation,53,,
8,Feb 21 2023 2:53PM,257877,10,ISDIN Corporation,6,,
9,Feb 21 2023 2:51PM,257874,20,"Emersa Waterbox, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation,14,,
1,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation,38,,
2,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation,34,,
3,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc.",17,,
4,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc.",6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation,14.0,NaN,NaN
1,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation,38.0,NaN,NaN
2,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation,34.0,NaN,NaN
3,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc.",17.0,NaN,NaN
4,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc.",6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 22 2023 8:30AM,257885,10,ISDIN Corporation,14.0,0.0,0.0
1,Feb 22 2023 8:30AM,257879,10,ISDIN Corporation,38.0,0.0,0.0
2,Feb 22 2023 8:30AM,257878,10,ISDIN Corporation,34.0,0.0,0.0
3,Feb 22 2023 8:30AM,257881,10,"Methapharm, Inc.",17.0,0.0,0.0
4,Feb 22 2023 8:30AM,257880,10,"Methapharm, Inc.",6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,9540137,480.0,11.0,4.0
12,257810,1.0,0.0,0.0
15,257739,1.0,5.0,13.0
19,1031457,3.0,1.0,0.0
20,515724,2.0,0.0,0.0
21,515648,1.0,1.0,0.0
22,515705,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,9540137,480.0,11.0,4.0
1,12,257810,1.0,0.0,0.0
2,15,257739,1.0,5.0,13.0
3,19,1031457,3.0,1.0,0.0
4,20,515724,2.0,0.0,0.0
5,21,515648,1.0,1.0,0.0
6,22,515705,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,480.0,11.0,4.0
1,12,1.0,0.0,0.0
2,15,1.0,5.0,13.0
3,19,3.0,1.0,0.0
4,20,2.0,0.0,0.0
5,21,1.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,480.0
1,12,Released,1.0
2,15,Released,1.0
3,19,Released,3.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,4.0,0.0,13.0,0.0,0.0,0.0,0.0
Executing,11.0,0.0,5.0,1.0,0.0,1.0,0.0
Released,480.0,1.0,1.0,3.0,2.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,4.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,11.0,0.0,5.0,1.0,0.0,1.0,0.0
2,Released,480.0,1.0,1.0,3.0,2.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,4.0,0.0,13.0,0.0,0.0,0.0,0.0
1,Executing,11.0,0.0,5.0,1.0,0.0,1.0,0.0
2,Released,480.0,1.0,1.0,3.0,2.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()